In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("coffee_cognition_synthetic_dataset.csv")

In [3]:
df.head()

,Participant_ID,Age,Gender,Brewing_Method,Time_of_Day,Caffeine_mg,Sleep_Hours,Sleep_Quality_Score,Stress_Level,Physical_Activity_Level,Stroop_Reaction_Time_ms,N_Back_Accuracy,PVT_Reaction_Time_ms,Focus_Level
0,P1000,56,Female,Drip,Morning,128.698060,7.581691,7.137768,3.591877,8.662958,510.566306,0.980000,281.727137,0.978034
1,P1001,46,Female,Pour Over,Morning,144.738069,9.000000,7.003487,3.592791,7.523608,487.581412,0.980000,191.029917,0.247670
2,P1002,32,Female,Drip,Afternoon,179.502409,8.421774,6.733141,3.361499,8.564198,505.235389,0.980000,218.402429,0.238690
3,P1003,25,Female,Drip,Morning,152.313091,7.356962,10.000000,1.626963,6.233739,507.385556,0.980000,204.185196,1.000000
4,P1004,38,Female,Pour Over,Afternoon,69.942719,6.104359,6.063791,4.539595,6.633885,659.780967,0.705424,289.089974,0.590769


In [5]:
coffee_features = ["Brewing_Method", "Caffeine_mg", "Time_of_Day"]

lifestyle_features = ["Sleep_Hours", "Sleep_Quality_Score", "Stress_Level", "Physical_Activity_Level"]

demographic_features = ["Age", "Gender"]

targets = ["Stroop_Reaction_Time_ms", "PVT_Reaction_Time_ms", "N_Back_Accuracy", "Focus_Level"]

In [8]:
df_encoded = pd.get_dummies(
    df,
    columns = ["Brewing_Method", "Time_of_Day", "Gender"],
    drop_first = True
)

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numeric_features = [
    "Age",
    "Caffeine_mg",
    "Sleep_Hours",
    "Sleep_Quality_Score",
    "Stress_Level",
    "Physical_Activity_Level"
]

df_encoded[numeric_features] = scaler.fit_transform(df_encoded[numeric_features])

In [10]:
df_encoded["Caffeine_x_SleepQuality"] = (
    df_encoded["Caffeine_mg"] * df_encoded["Sleep_Quality_Score"]
)

In [11]:
df_encoded.head()

,Participant_ID,Age,Caffeine_mg,Sleep_Hours,Sleep_Quality_Score,Stress_Level,Physical_Activity_Level,Stroop_Reaction_Time_ms,N_Back_Accuracy,PVT_Reaction_Time_ms,Focus_Level,Brewing_Method_Drip,Brewing_Method_Espresso,Brewing_Method_French Press,Brewing_Method_Pour Over,Time_of_Day_Evening,Time_of_Day_Morning,Gender_Male,Caffeine_x_SleepQuality
0,P1000,1.365011,-0.476625,0.571583,0.428717,-0.999060,1.396815,510.566306,0.980000,281.727137,0.978034,True,False,False,False,False,True,False,-0.204337
1,P1001,0.565590,-0.230287,1.727296,0.336843,-0.998570,0.790189,487.581412,0.980000,191.029917,0.247670,False,False,False,True,False,True,False,-0.077571
2,P1002,-0.553599,0.303614,1.256127,0.151875,-1.122568,1.344232,505.235389,0.980000,218.402429,0.238690,True,False,False,False,False,False,False,0.046112
3,P1003,-1.113193,-0.113952,0.388461,2.387025,-2.052473,0.103423,507.385556,0.980000,204.185196,1.000000,True,False,False,False,False,True,False,-0.272006
4,P1004,-0.073946,-1.378975,-0.632226,-0.306087,-0.490977,0.316473,659.780967,0.705424,289.089974,0.590769,False,False,False,True,False,False,False,0.422086


In [12]:
df_encoded["Caffeine_x_Stress"] = (
    df_encoded["Caffeine_mg"] * df_encoded["Stress_Level"]
)

In [13]:
if "Time_of_Day_Evening" in df_encoded.columns:
    df_encoded["Caffeine_x_Time_of_Day"] = (
        df_encoded["Caffeine_mg"] * df_encoded["Time_of_Day_Evening"]
    )

In [15]:
x = df_encoded.drop(columns=targets)
y_stroop = df_encoded["Stroop_Reaction_Time_ms"]
y_pvt = df_encoded["PVT_Reaction_Time_ms"]
y_nback = df_encoded["N_Back_Accuracy"]
y_focus = df_encoded["Focus_Level"]

In [16]:
x.shape

(280, 17)

In [18]:
df_encoded.to_csv("coffee_cognition_processed.csv", index=False)